https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

### Data Preparation

In [52]:
import string

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r', encoding='utf-8')
    # read all text
    text = file.read()
    # print(file)
    # close the file
    file.close()
    return text

In [53]:
#import io

#with io.open(filename, encoding='utf-8', mode='r') as file:
#    for line in file:
#        process(line)

In [54]:
# load document
in_filename = 'republic_clean.txt'
doc = load_doc(in_filename)
print(doc[:200])

﻿BOOK I.


I went down yesterday to the Piraeus with Glaucon the son of Ariston,
that I might offer up my prayers to the goddess (Bendis, the Thracian
Artemis.); and also because I wanted to see in wh


In [55]:
# Clean Text
#import string
 
# turn a doc into clean tokens
def clean_doc(doc):
    # replace '--' with a space ' '
    doc = doc.replace('--', ' ')
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # make lower case
    tokens = [word.lower() for word in tokens]
    return tokens

In [56]:
# clean document
tokens = clean_doc(doc)
print(tokens[:200])
print('Total Tokens: %d' % len(tokens))
print('Unique Tokens: %d' % len(set(tokens)))

['i', 'i', 'went', 'down', 'yesterday', 'to', 'the', 'piraeus', 'with', 'glaucon', 'the', 'son', 'of', 'ariston', 'that', 'i', 'might', 'offer', 'up', 'my', 'prayers', 'to', 'the', 'goddess', 'bendis', 'the', 'thracian', 'artemis', 'and', 'also', 'because', 'i', 'wanted', 'to', 'see', 'in', 'what', 'manner', 'they', 'would', 'celebrate', 'the', 'festival', 'which', 'was', 'a', 'new', 'thing', 'i', 'was', 'delighted', 'with', 'the', 'procession', 'of', 'the', 'inhabitants', 'but', 'that', 'of', 'the', 'thracians', 'was', 'equally', 'if', 'not', 'more', 'beautiful', 'when', 'we', 'had', 'finished', 'our', 'prayers', 'and', 'viewed', 'the', 'spectacle', 'we', 'turned', 'in', 'the', 'direction', 'of', 'the', 'city', 'and', 'at', 'that', 'instant', 'polemarchus', 'the', 'son', 'of', 'cephalus', 'chanced', 'to', 'catch', 'sight', 'of', 'us', 'from', 'a', 'distance', 'as', 'we', 'were', 'starting', 'on', 'our', 'way', 'home', 'and', 'told', 'his', 'servant', 'to', 'run', 'and', 'bid', 'us', '

In [57]:
# organize into sequences of tokens
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
    # select sequence of tokens
    seq = tokens[i-length:i]
    # convert into a line
    line = ' '.join(seq)
    # store
    sequences.append(line)
print('Total Sequences: %d' % len(sequences))

Total Sequences: 117290


In [58]:
# save tokens to file, one dialog per line
def save_doc(lines, republic_sequences):
    data = '\n'.join(lines)
    file = open(republic_sequences, 'w')
    file.write(data)
    file.close()

In [59]:
# save sequences to file
out_filename = 'republic_sequences.txt'
save_doc(sequences, out_filename)
doc_sequences = load_doc(out_filename)

#i = 1
#with open('republic_sequences.txt', 'r') as f:
#    for lines in f:
#        if i >= 500:
#            break
#        else:
#            print(doc_sequences[i:51])
#            i = i + 51

#with open('republic_sequences.txt', 'r') as f:
#    for line in f:
#        print(doc_sequences[:8])

### Train Language Model

 
    We will use an Embedding Layer to learn the representation of words, and a Long Short-Term Memory (LSTM) recurrent neural network to learn to predict words based on their context

In [60]:
from numpy import array
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

In [61]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [62]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1

In [63]:
# separate into input and output
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]
seq_length

50

Обученное встраивание должно знать размер словаря и длину входных последовательностей, как обсуждалось ранее. У него также есть параметр, указывающий, сколько измерений будет использоваться для представления каждого слова. То есть размер векторного пространства вложения.

Распространенными значениями являются 50, 100 и 300. Мы будем использовать здесь 50, но рассмотрите возможность тестирования меньших или больших значений.

Мы будем использовать два скрытых слоя LSTM по 100 ячеек памяти каждый. Больше ячеек памяти и более глубокая сеть могут дать лучшие результаты.

Плотный полностью связанный слой со 100 нейронами соединяется со скрытыми слоями LSTM для интерпретации признаков, извлеченных из последовательности. Выходной слой предсказывает следующее слово как единый вектор размера словаря с вероятностью для каждого слова в словаре. Функция активации softmax используется для обеспечения того, чтобы выходные данные имели характеристики нормализованных вероятностей.

In [64]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 50)            366200    
                                                                 
 lstm_2 (LSTM)               (None, 50, 100)           60400     
                                                                 
 lstm_3 (LSTM)               (None, 100)               80400     
                                                                 
 dense_2 (Dense)             (None, 100)               10100     
                                                                 
 dense_3 (Dense)             (None, 7324)              739724    
                                                                 
Total params: 1,256,824
Trainable params: 1,256,824
Non-trainable params: 0
_________________________________________________________________
None


Затем модель компилируется с указанием категориальной потери перекрестной энтропии, необходимой для соответствия модели. Технически модель изучает многоклассовую классификацию, и это подходящая функция потерь для такого типа задач. Используется эффективная реализация Адама для мини-пакетного градиентного спуска, и оценивается точность модели.

Наконец, модель соответствует данным для 100 обучающих эпох со скромным размером пакета 128 для ускорения процесса.

Обучение может занять несколько часов на современном оборудовании без графических процессоров. Вы можете ускорить его, увеличив размер партии и/или уменьшив количество периодов обучения.

Во время обучения вы увидите сводку производительности, включая потери и точность, оцененные на основе обучающих данных в конце каждого пакетного обновления.

##### Примечание. Ваши результаты могут отличаться из-за стохастического характера алгоритма или процедуры оценки, а также из-за различий в численной точности. Попробуйте запустить пример несколько раз и сравните средний результат.

##### Вы получите разные результаты, но, возможно, точность предсказания следующего слова в последовательности составит чуть более 50 %, что неплохо. Мы не стремимся к 100% точности (например, модель, которая запоминает текст), а скорее к модели, которая улавливает суть текста.

In [20]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
917/917 [==============================] - 102s 108ms/step - loss: 6.1362 - accuracy: 0.0751
Epoch 2/100
917/917 [==============================] - 96s 105ms/step - loss: 5.6759 - accuracy: 0.1094
Epoch 3/100
917/917 [==============================] - 97s 105ms/step - loss: 5.4402 - accuracy: 0.1331
Epoch 4/100
917/917 [==============================] - 96s 105ms/step - loss: 5.2810 - accuracy: 0.1466
Epoch 5/100
917/917 [==============================] - 96s 105ms/step - loss: 5.1669 - accuracy: 0.1554
Epoch 6/100
917/917 [==============================] - 97s 106ms/step - loss: 5.0691 - accuracy: 0.1630
Epoch 7/100
917/917 [==============================] - 96s 105ms/step - loss: 4.9802 - accuracy: 0.1689
Epoch 8/100
917/917 [==============================] - 97s 106ms/step - loss: 4.8978 - accuracy: 0.1739
Epoch 9/100
917/917 [==============================] - 100s 109ms/step - loss: 4.8201 - accuracy: 0.1774
Epoch 10/100
917/917 [==============================] - 98s 10

In [21]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

In [22]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=250)

Epoch 1/250
917/917 [==============================] - 121s 124ms/step - loss: 2.7794 - accuracy: 0.3885
Epoch 2/250
917/917 [==============================] - 106s 116ms/step - loss: 2.6854 - accuracy: 0.4049
Epoch 3/250
917/917 [==============================] - 106s 116ms/step - loss: 2.6693 - accuracy: 0.4089
Epoch 4/250
917/917 [==============================] - 106s 115ms/step - loss: 2.6540 - accuracy: 0.4102
Epoch 5/250
917/917 [==============================] - 106s 115ms/step - loss: 2.6385 - accuracy: 0.4142
Epoch 6/250
917/917 [==============================] - 105s 115ms/step - loss: 2.6278 - accuracy: 0.4152
Epoch 7/250
917/917 [==============================] - 105s 114ms/step - loss: 2.6148 - accuracy: 0.4175
Epoch 8/250
917/917 [==============================] - 105s 114ms/step - loss: 2.6013 - accuracy: 0.4215
Epoch 9/250
917/917 [==============================] - 106s 115ms/step - loss: 2.5895 - accuracy: 0.4216
Epoch 10/250
917/917 [==============================] -

917/917 [==============================] - 201s 219ms/step - loss: 1.5828 - accuracy: 0.6200
Epoch 155/250
917/917 [==============================] - 204s 222ms/step - loss: 1.5683 - accuracy: 0.6242
Epoch 156/250
917/917 [==============================] - 204s 223ms/step - loss: 1.5703 - accuracy: 0.6223
Epoch 157/250
917/917 [==============================] - 203s 221ms/step - loss: 1.5676 - accuracy: 0.6227
Epoch 158/250
917/917 [==============================] - 205s 223ms/step - loss: 1.5506 - accuracy: 0.6261
Epoch 159/250
917/917 [==============================] - 204s 223ms/step - loss: 1.5511 - accuracy: 0.6264
Epoch 160/250
917/917 [==============================] - 206s 224ms/step - loss: 1.5558 - accuracy: 0.6247
Epoch 161/250
917/917 [==============================] - 206s 225ms/step - loss: 1.5502 - accuracy: 0.6271
Epoch 162/250
917/917 [==============================] - 207s 225ms/step - loss: 1.5424 - accuracy: 0.6285
Epoch 163/250
917/917 [============================

In [23]:
# save the model to file
model.save('modelx350.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

### Use Language Model

In [65]:
# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, 'r')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text
 
# load cleaned text sequences
in_filename = 'republic_sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')

Нам нужен текст, чтобы мы могли выбрать исходную последовательность в качестве входных данных для модели для создания новой последовательности текста.

Модель потребует 50 слов в качестве входных данных.

Позже нам нужно будет указать ожидаемую длину ввода. Мы можем определить это по входным последовательностям, вычислив длину одной строки загруженных данных и вычтя 1 для ожидаемого выходного слова, которое также находится в той же строке.

In [66]:
seq_length = len(lines[0].split()) - 1
seq_length

50

In [77]:
from tensorflow.keras import models    

# load the model
model = models.load_model('modelx350.h5')
print(model.summary())
model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            366200    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7324)              739724    
                                                                 
Total params: 1,256,824
Trainable params: 1,256,824
Non-trainable params: 0
_________________________________________________________________
None


### Generate Text

Первым шагом в генерации текста является подготовка начального ввода.

Для этого мы выберем случайную строку текста из входного текста. После выбора мы распечатаем его, чтобы иметь некоторое представление о том, что было использовано.

In [25]:
import numpy as np
from pickle import dump
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [68]:
from random import randint

# select a seed text
seed_text = lines[randint(0,len(lines))]
print(seed_text + '\n')

replied for i suppose that you mean to exclude mere uninstructed courage such as that of a wild beast or of a in your opinion is not the courage which the law ordains and ought to have another name most certainly then i may infer courage to be such as you



Далее мы можем генерировать новые слова по одному.

Во-первых, начальный текст должен быть закодирован в целые числа с использованием того же токенизатора, который мы использовали при обучении модели.

In [69]:
encoded = tokenizer.texts_to_sequences([seed_text])[0]

Модель может предсказать следующее слово напрямую, вызвав model.predict_classes(), которая вернет индекс слова с наибольшей вероятностью.

Синтакис устарел (фукция удалена) с версии tansorflow 2.6, замена:

#predict_x=model.predict(X_test) 
#classes_x=np.argmax(predict_x,axis=1)

#y_predict = np.argmax(model.predict(x_test), axis=-1)

#predictions = (model.predict(x_test) > 0.5).astype("int32")

In [71]:
# predict probabilities for each word
# yhat = model.predict_classes(encoded, verbose=0)
#yhat=model.predict(encoded)
#classes_x=np.argmax(yhat,axis=1)

#y_predict = np.argmax(model.predict(x_test), axis=-1)

#yhat = (model.predict(encoded) > 0.5).astype("int32")

#predictions = np.argmax(model.predict(encoded),axis=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 50)            366200    
                                                                 
 lstm (LSTM)                 (None, 50, 100)           60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 7324)              739724    
                                                                 
Total params: 1,256,824
Trainable params: 1,256,824
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
out_word = ''
for word, index in tokenizer.word_index.items():
    if index == yhat:
        out_word = word
        break

NameError: name 'yhat' is not defined

In [46]:
# generate a sequence from a language model
from keras.preprocessing.sequence import pad_sequences

def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        #yhat = model.predict_classes(encoded, verbose=0)
        
        yhat=model.predict(encoded)
        classes_x=np.argmax(yhat,axis=1)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [47]:
# generate new text
generated = generate_seq(model, tokenizer, seq_length, seed_text, 50)
print(generated)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()